[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremylongshore/claude-code-plugins-plus-skills/blob/main/tutorials/plugins/01-what-is-plugin.ipynb)

# What is a Claude Code Plugin?

**Learning Path**: Skills → **Plugins** → Orchestration  
**Level**: Beginner  
**Time**: 25 minutes  
**Prerequisites**: [Skills tutorials](../skills/01-what-is-skill.ipynb) recommended but not required

---

## What You'll Learn

1. ✅ **What plugins are** - Containers for capabilities
2. ✅ **Plugin types** - Instruction, MCP, Agent-equipped
3. ✅ **When to use each** - Decision framework
4. ✅ **Real examples** - From the marketplace (258 plugins)
5. ✅ **Plugin discovery** - How Claude finds and loads plugins
6. ✅ **Marketplace browsing** - Interactive catalog exploration

---

## The Problem: Scattered Knowledge

**Without plugins**:
```
You: "Claude, analyze this Python code for security issues."
     [Paste 50 lines of security checklist every time]
     [Repeat for every project]
```

**With plugins**:
```
You: "Analyze this code for security issues."
Claude: [Automatically uses python-security-scanner plugin]
        [Already knows OWASP Top 10, PEP 8, best practices]
```

**The difference**: Plugins package expertise into reusable, discoverable units.

In [ ]:
# What is a plugin?
plugin_definition = {
    "what": "Container for related skills, commands, and integrations",
    "format": "Directory with plugin.json + optional components",
    "discovery": "Automatic - Claude scans marketplace catalogs",
    "persistence": "Installed once, works across all conversations",
    "portability": "Shareable via GitHub, npm, or marketplace"
}

print("WHAT IS A CLAUDE CODE PLUGIN?")
print("=" * 60)
for aspect, explanation in plugin_definition.items():
    print(f"{aspect.upper()}: {explanation}")

print("\n💡 Think of plugins as npm packages for Claude!")

## Plugin vs Skill: What's the Difference?

| Aspect | Skill | Plugin |
|--------|-------|--------|
| **Purpose** | Single capability | Container for multiple capabilities |
| **Structure** | One SKILL.md file | Directory with plugin.json + components |
| **Contents** | Instructions + tool auth | Skills + commands + agents + MCP |
| **Scope** | Focused workflow | Related set of workflows |
| **Example** | `git-commit-helper` | `git-pro` (commit, rebase, merge skills) |

### Mental Model

```
Plugin (Container)
├── Skill 1: git-commit-helper
├── Skill 2: git-rebase-wizard
├── Skill 3: git-merge-resolver
├── Command: /git-status
└── README.md
```

**Analogy**: 
- **Skill** = Single function in a library
- **Plugin** = The entire library (collection of functions)

In [ ]:
# Skill vs Plugin comparison
def compare_skill_plugin():
    examples = [
        {
            "type": "Standalone Skill",
            "structure": ".claude/skills/test-generator/SKILL.md",
            "contents": "1 SKILL.md file",
            "use_case": "Simple, focused task"
        },
        {
            "type": "Plugin with Skills",
            "structure": "plugins/testing-pro/",
            "contents": "plugin.json + 5 skills + 3 commands + README",
            "use_case": "Comprehensive testing toolkit"
        }
    ]
    
    print("SKILL vs PLUGIN")
    print("=" * 60)
    for ex in examples:
        print(f"\n{ex['type'].upper()}:")
        print(f"  Location: {ex['structure']}")
        print(f"  Contents: {ex['contents']}")
        print(f"  Best for: {ex['use_case']}")
    
    print("\n💡 Use skills for single tasks, plugins for related toolkits!")

compare_skill_plugin()

## Plugin Types: The Three Categories

### 1. Instruction Plugins (98% of marketplace)

**What**: Markdown files with YAML frontmatter (no code execution)

**Contains**:
- Skills (SKILL.md files)
- Commands (slash commands)
- Agents (specialized subagents)
- Documentation

**Example**: `python-security-scanner`
```
plugins/security/python-security-scanner/
├── .claude-plugin/plugin.json
├── skills/
│   ├── owasp-scanner/SKILL.md
│   └── dependency-checker/SKILL.md
├── commands/
│   └── security-audit.md
└── README.md
```

**Pros**: Easy to create, no dependencies, fast to load  
**Cons**: No external tool access, limited to Claude's built-in tools

---

### 2. MCP Server Plugins (6 in marketplace)

**What**: TypeScript/Node.js servers implementing Model Context Protocol

**Contains**:
- Custom tools (functions Claude can call)
- External API integrations
- Database connections
- File system operations

**Example**: `sqlite-mcp`
```typescript
// src/index.ts
server.tool("query_database", async (params) => {
  const result = await db.query(params.sql);
  return { rows: result };
});
```

**Pros**: Full programming capability, external integrations  
**Cons**: Requires TypeScript, more complex, needs building

---

### 3. Hybrid Plugins (Best of Both)

**What**: Instruction plugin + MCP server

**Contains**:
- Skills that guide usage (instruction layer)
- MCP tools for execution (code layer)
- Commands for common operations

**Example**: `database-pro`
- Skill: "How to design schemas"
- MCP tool: `execute_migration(sql)`
- Command: `/db-migrate`

**Pros**: Best UX (guidance + power)  
**Cons**: Most complex to maintain

In [ ]:
# Plugin type decision tree
def recommend_plugin_type(requirements):
    """
    Recommend plugin type based on requirements.
    
    Requirements dict:
        - needs_external_api: bool
        - needs_database: bool
        - complexity: 'simple' | 'medium' | 'complex'
        - user_guidance: bool
    """
    needs_code = requirements.get('needs_external_api') or requirements.get('needs_database')
    needs_guidance = requirements.get('user_guidance')
    
    if needs_code and needs_guidance:
        return {
            "type": "Hybrid Plugin",
            "reason": "Need both user guidance (skills) and code execution (MCP)",
            "example": "database-pro, api-client-generator"
        }
    elif needs_code:
        return {
            "type": "MCP Server Plugin",
            "reason": "External integration required, minimal guidance needed",
            "example": "sqlite-mcp, postgres-connector"
        }
    else:
        return {
            "type": "Instruction Plugin",
            "reason": "Pure guidance, uses Claude's built-in tools only",
            "example": "python-security-scanner, git-workflow-helper"
        }

# Test cases
test_scenarios = [
    {
        "name": "Code review plugin",
        "needs_external_api": False,
        "needs_database": False,
        "complexity": "medium",
        "user_guidance": True
    },
    {
        "name": "Database migration tool",
        "needs_external_api": False,
        "needs_database": True,
        "complexity": "complex",
        "user_guidance": True
    },
    {
        "name": "SQLite browser",
        "needs_external_api": False,
        "needs_database": True,
        "complexity": "simple",
        "user_guidance": False
    }
]

print("PLUGIN TYPE RECOMMENDATION")
print("=" * 60)
for scenario in test_scenarios:
    name = scenario.pop('name')
    result = recommend_plugin_type(scenario)
    print(f"\n{name}:")
    print(f"  → {result['type']}")
    print(f"  Why: {result['reason']}")
    print(f"  Like: {result['example']}")

## Real Examples from the Marketplace

### Top Instruction Plugins (258 total)

| Plugin | Category | Skills | Description |
|--------|----------|--------|-------------|
| `python-security-scanner` | Security | 3 | OWASP analysis, dependency checking |
| `git-workflow-helper` | DevOps | 5 | Commit, rebase, merge, conflict resolution |
| `taskwarrior-integration` | Productivity | 2 | Task management with Taskwarrior CLI |
| `yaml-master-agent` | Productivity | 4 | YAML validation, linting, conversion |
| `vertex-ai-media-master` | AI/ML | 6 | Video, audio, image generation |

### MCP Server Plugins (6 total)

| Plugin | Tools | Description |
|--------|-------|-------------|
| `sqlite-mcp` | 3 | Query, insert, update SQLite databases |
| `postgres-connector` | 5 | Full PostgreSQL integration |
| `api-client-generator` | 4 | Generate API clients from OpenAPI specs |

### Stats from Marketplace

- **Total plugins**: 258
- **Skills enabled**: 241
- **Categories**: 18
- **Top category**: Productivity (42 plugins)
- **Newest feature**: Agent Skills (240 equipped)

In [ ]:
import json
from pathlib import Path

# Load marketplace catalog
marketplace_path = Path('.claude-plugin/marketplace.json')

def browse_marketplace(category_filter=None, limit=10):
    """
    Browse marketplace catalog interactively.
    
    Args:
        category_filter: Optional category to filter by
        limit: Max plugins to show
    """
    try:
        with open(marketplace_path) as f:
            catalog = json.load(f)
    except FileNotFoundError:
        print(f"❌ Marketplace catalog not found at {marketplace_path}")
        return
    
    metadata = catalog.get('metadata', {})
    plugins = catalog.get('plugins', [])
    
    print("MARKETPLACE OVERVIEW")
    print("=" * 60)
    print(f"Total Plugins: {metadata.get('totalPlugins', len(plugins))}")
    print(f"Skills Enabled: {metadata.get('skillsEnabled', 'N/A')}")
    print(f"Last Updated: {metadata.get('lastUpdated', 'N/A')}")
    print(f"Version: {metadata.get('version', 'N/A')}")
    
    # Filter plugins
    if category_filter:
        filtered = [p for p in plugins if p.get('category') == category_filter]
        print(f"\nFiltered by category: {category_filter} ({len(filtered)} plugins)")
    else:
        filtered = plugins
    
    # Show top plugins
    print(f"\nTOP {min(limit, len(filtered))} PLUGINS:")
    print("-" * 60)
    
    for i, plugin in enumerate(filtered[:limit], 1):
        name = plugin.get('name', 'Unknown')
        category = plugin.get('category', 'uncategorized')
        version = plugin.get('version', '0.0.0')
        desc = plugin.get('description', 'No description')[:80]
        
        print(f"\n{i}. {name} (v{version})")
        print(f"   Category: {category}")
        print(f"   {desc}...")
    
    return filtered

# Browse all plugins
browse_marketplace(limit=5)

# Try filtering by category
print("\n\n" + "=" * 60)
browse_marketplace(category_filter="security", limit=3)

## How Claude Discovers Plugins

### Discovery Priority (4 Locations)

| Location | Scope | Priority | Installed By |
|----------|-------|----------|-------------|
| `~/.claude/plugins/` | Personal | 1 (lowest) | Manual install |
| `.claude/plugins/` | Project | 2 | Project team |
| Marketplace | Shared | 3 | `/plugin marketplace add` |
| Built-in | Platform | 4 (highest) | Claude Code core |

### Installation Methods

**1. Manual (Copy directory)**
```bash
cp -r my-plugin/ ~/.claude/plugins/
```

**2. Git Clone**
```bash
cd ~/.claude/plugins/
git clone https://github.com/user/plugin-name.git
```

**3. Marketplace (Recommended)**
```bash
/plugin marketplace add jeremylongshore/claude-code-plugins
```

**4. NPM (Future)**
```bash
npm install -g @claude-plugins/security-scanner
```

In [ ]:
import os
from pathlib import Path

def check_plugin_locations():
    """
    Check all plugin discovery locations.
    """
    locations = [
        {
            "path": Path.home() / ".claude" / "plugins",
            "name": "Personal plugins",
            "priority": 1
        },
        {
            "path": Path.cwd() / ".claude" / "plugins",
            "name": "Project plugins",
            "priority": 2
        },
        {
            "path": Path.cwd() / "plugins",
            "name": "Marketplace plugins",
            "priority": 3
        }
    ]
    
    print("PLUGIN DISCOVERY LOCATIONS")
    print("=" * 60)
    
    for loc in locations:
        exists = loc["path"].exists()
        status = "✅ Found" if exists else "❌ Not found"
        
        print(f"\n{loc['name']} (Priority {loc['priority']}):")
        print(f"  Path: {loc['path']}")
        print(f"  Status: {status}")
        
        if exists and loc["path"].is_dir():
            plugin_dirs = [d for d in loc["path"].iterdir() 
                          if d.is_dir() and (d / ".claude-plugin" / "plugin.json").exists()]
            print(f"  Plugins found: {len(plugin_dirs)}")
            if plugin_dirs:
                for plugin in plugin_dirs[:3]:
                    print(f"    - {plugin.name}")
    
    print("\n💡 Higher priority wins on name conflicts!")

check_plugin_locations()

## When to Create a Plugin vs Skill

### Create a Standalone Skill When:

- ✅ Single, focused capability
- ✅ No related workflows
- ✅ Quick prototyping
- ✅ Personal use only

**Example**: `generate-test-data` skill for creating mock JSON

---

### Create a Plugin When:

- ✅ Multiple related skills
- ✅ Slash commands needed
- ✅ Sharing with team/community
- ✅ Versioning and updates important
- ✅ Documentation required

**Example**: `testing-toolkit` plugin with:
- Skill: `generate-test-data`
- Skill: `mock-api-responses`
- Skill: `fixture-generator`
- Command: `/test-setup`
- README with examples

---

### Create an MCP Plugin When:

- ✅ External API integration
- ✅ Database access
- ✅ File system operations
- ✅ Custom computation
- ✅ Real-time data

**Example**: `stripe-mcp` for payment processing

---

### Decision Tree

```
Need external APIs/databases?
├─ Yes → MCP Plugin (or Hybrid)
└─ No → Multiple related capabilities?
         ├─ Yes → Instruction Plugin
         └─ No → Standalone Skill
```

In [ ]:
# Interactive decision helper
def should_i_create_plugin():
    """
    Interactive helper to decide skill vs plugin.
    """
    questions = [
        ("Do you need external API access?", "mcp"),
        ("Do you need database connections?", "mcp"),
        ("Do you have 3+ related capabilities?", "plugin"),
        ("Will you share this with others?", "plugin"),
        ("Do you need slash commands?", "plugin"),
    ]
    
    # Simulate answers (in real notebook, use input())
    answers = {
        questions[0]: False,
        questions[1]: False,
        questions[2]: True,
        questions[3]: True,
        questions[4]: True,
    }
    
    mcp_score = sum(1 for (q, tag) in questions if answers.get(q, False) and tag == "mcp")
    plugin_score = sum(1 for (q, tag) in questions if answers.get(q, False) and tag == "plugin")
    
    print("SHOULD I CREATE A PLUGIN?")
    print("=" * 60)
    
    for (question, tag) in questions:
        answer = "Yes" if answers.get(question, False) else "No"
        print(f"{question} {answer}")
    
    print(f"\nMCP score: {mcp_score}/2")
    print(f"Plugin score: {plugin_score}/3")
    
    if mcp_score >= 1:
        recommendation = "🔧 MCP Server Plugin"
        reason = "You need code execution for external integrations"
    elif plugin_score >= 2:
        recommendation = "📦 Instruction Plugin"
        reason = "Multiple capabilities warrant a plugin container"
    else:
        recommendation = "📄 Standalone Skill"
        reason = "Simple, focused capability doesn't need plugin overhead"
    
    print(f"\n✨ RECOMMENDATION: {recommendation}")
    print(f"   Why: {reason}")
    
    return recommendation

should_i_create_plugin()

## Key Takeaways

### What You Learned

1. ✅ **Plugins are containers** - Package related skills, commands, agents
2. ✅ **Three plugin types** - Instruction (98%), MCP (2%), Hybrid (best UX)
3. ✅ **Discovery hierarchy** - Personal → Project → Marketplace → Built-in
4. ✅ **258 plugins available** - Real examples from marketplace
5. ✅ **Decision framework** - When to create plugin vs skill
6. ✅ **Interactive browsing** - Explore catalog programmatically

### Quick Reference

**Create a Skill when**: Single capability, personal use  
**Create a Plugin when**: Multiple capabilities, sharing, versioning  
**Create an MCP Plugin when**: External APIs, databases, file system

### Marketplace Stats

- 258 total plugins
- 241 with Agent Skills
- 18 categories
- 6 MCP server plugins

---

## Next Steps

1. **[02-plugin-structure.ipynb](02-plugin-structure.ipynb)** - Dive into plugin anatomy
2. **[03-build-your-first-plugin.ipynb](03-build-your-first-plugin.ipynb)** - Hands-on creation
3. **[04-mcp-server-plugins.ipynb](04-mcp-server-plugins.ipynb)** - Advanced MCP
4. **Browse marketplace** - `/plugin marketplace list`

---

*Enterprise Standards Compliant • Version 1.0.0 • MIT License*